<a href="https://colab.research.google.com/github/kameda-yoshinari/IMISToolExeA2022/blob/main/600/pytorch_advanced-revised/7_nlp_sentiment_transformer/GC7_5_IMDb_Dataset_DataLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.5 IMDb（Internet Movie Database）からDataLoaderを作成

- 本ファイルでは、IMDb（Internet Movie Database）のデータを使用して、感情分析（0：ネガティブ、1：ポジティブ）を2値クラス分類するためのDatasetとDataLoaderを作成します。


※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.5 学習目標

1.	テキスト形式のファイルデータからtsvファイルを作成し、torchtext用のDataLoaderを作成できるようになる

---

# Google Colab

In [ ]:
!echo "Change to the JST notation."
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime

Change to the JST notation.


In [ ]:
!echo "Start mounting your Google Drive."
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/
!echo "Move to the working directory."
%cd IMIS_Tool-A/Work600/
!ls -l

Start mounting your Google Drive.
Mounted at /content/drive
/content/drive/My Drive
Move to the working directory.
/content/drive/My Drive/IMIS_Tool-A/Work600
total 4
drwx------ 2 root root 4096 Aug  7 22:01 pytorch_advanced


---
# 共通準備

"pytorch_advanced" folder should be ready before you come here.

In [ ]:
# Skip this if you have already issued git in advance. 
# If you come here by way of 600-PyTorchADL.ipynb, 
# you should skip the git command (as you have already issued in 600).  
# If you run git when pytorch_advanced already exists, git tells the error and clone won't be made.

#!git clone https://github.com/YutaroOgawa/pytorch_advanced.git

import os
if os.path.exists("/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced"):
    print("OK. Alreadly git cloned. You can go.")
else:
    print("You'd better go back to the first 600-PyTorchADL.ipynb")

OK. Alreadly git cloned. You can go.


In [ ]:
!ls

pytorch_advanced


In [ ]:
%cd "pytorch_advanced"

/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced


In [ ]:
!ls

1_image_classification	 7_nlp_sentiment_transformer
2_objectdetection	 8_nlp_sentiment_bert
3_semantic_segmentation  9_video_classification_eco
4_pose_estimation	 etc
5_gan_generation	 LICENSE
6_gan_anomaly_detection  README.md


In [ ]:
%cd "7_nlp_sentiment_transformer"

/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced/7_nlp_sentiment_transformer


In [ ]:
!ls

7-1_Tokenizer.ipynb		   7-7_transformer_training_inference.ipynb
7-2_torchtext.ipynb		   data
7-4_vectorize.ipynb		   make_folders_and_data_downloads.ipynb
7-5_IMDb_Dataset_DataLoader.ipynb  utils
7-6_Transformer.ipynb


---
# Extraction of aclImdb (GC7-0)


In [ ]:
# local drive on GC is very, very fast.
!mkdir -p /root/data/
%cd /root/data/
!tar xfz '/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced/7_nlp_sentiment_transformer/data/aclImdb_v1.tar.gz'

# make a symbolic link at the working directory on google drive.
%cd '/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced/7_nlp_sentiment_transformer/'
!rm -f data/aclImdb
!ln -s /root/data/aclImdb data/aclImdb

!ls -ld data/aclImdb/

/root/data
/content/drive/My Drive/IMIS_Tool-A/Work600/pytorch_advanced/7_nlp_sentiment_transformer
drwxr-xr-x 4 7297 1000 4096 Jun 26  2011 data/aclImdb/


---
# Preparation for word separation (GC7-1)

This is the whole procedure of preparation of word separation.

In [ ]:
# janome, mecab, ipadic-neologd, and mecab-pytorch

!pip install janome 

# https://qiita.com/jun40vn/items/78e33e29dce3d50c2df1

%cd ~
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab 
!test -d mecab-ipadic-neologd || git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git 
!echo yes | ./mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n 
!pip install mecab-python3 

!test -f /usr/local/etc/mecabrc || ln -s /etc/mecabrc /usr/local/etc/mecabrc 

!echo "Dictionary path is : "
!echo `mecab-config --dicdir`"/mecab-ipadic-neologd"

%cd -

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.7 MB 1.1 MB/s 
/root
Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.58.0-2ubuntu3.19).
git is already the newest version (1:2.17.1-1ubuntu0.12).
sudo is already the newest version (1.8.21p2-3ubuntu1.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libmecab2 mecab-ipadic mecab-jumandic
  mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  file libmagic-mgc libmagic1 libmecab-dev libmecab2 mecab mecab-ipadic
  mecab-ipadic-utf8 mecab-jumandic mecab-jumandic-utf8 mecab-utils
  python-mecab
0 upgraded, 12 newly installed, 0 to remove and 19 not upgraded.
Need to get 29.3 MB of archives.

In [ ]:
# We use torchtext version 0.8 (and torch 1.7.1) that was popular when the book was published. See issue 148.
# https://github.com/YutaroOgawa/pytorch_advanced/issues/148
import warnings
warnings.simplefilter('ignore')

!pip uninstall -y torchtext torchvision torchaudio
!pip install torch==1.7.1+cu110 -f https://download.pytorch.org/whl/torch_stable.html
!pip install torchtext==0.8

Found existing installation: torchtext 0.13.0
Uninstalling torchtext-0.13.0:
  Successfully uninstalled torchtext-0.13.0
Found existing installation: torchvision 0.13.0+cu113
Uninstalling torchvision-0.13.0+cu113:
  Successfully uninstalled torchvision-0.13.0+cu113
Found existing installation: torchaudio 0.12.0+cu113
Uninstalling torchaudio-0.12.0+cu113:
  Successfully uninstalled torchaudio-0.12.0+cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.4 MB/s eta 0:03:59tcmalloc: large alloc 1147494400 bytes == 0x392a2000 @  0x7fae1819b615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████

---
# 1. IMDbデータセットをtsv形式に変換

Datasetをダウンロードします

※torchtextで標準でIMDbが使える関数があるのですが、今回は今後データセットが用意されていない場合でも対応できるように0から作ります。

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

5万件のデータ（train,testともに2.5万件）です。データidとrating（1-10）でファイル名が決まっています。

rateは10の方が良いです。4以下がnegative、7以上がpositiveにクラス分けされています。



In [ ]:
# tsv形式のファイルにします
import glob
import os
import io
import string


# 訓練データのtsvファイルを作成します

f = open('./data/IMDb_train.tsv', 'w')

path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)

path = './data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


In [ ]:
# テストデータの作成

f = open('./data/IMDb_test.tsv', 'w')

path = './data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = './data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # タブがあれば消しておきます
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


# 2. 前処理と単語分割の関数を定義

In [ ]:
import string
import re

# 以下の記号はスペースに置き換えます（カンマ、ピリオドを除く）。
# punctuationとは日本語で句点という意味です
print("区切り文字：", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 前処理


def preprocessing_text(text):
    # 改行コードを消去
    text = re.sub('<br />', '', text)

    # カンマ、ピリオド以外の記号をスペースに置換
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # ピリオドなどの前後にはスペースを入れておく
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# 分かち書き（今回はデータが英語で、簡易的にスペースで区切る）


def tokenizer_punctuation(text):
    return text.strip().split()


# 前処理と分かち書きをまとめた関数を定義
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 動作を確認します
print(tokenizer_with_preprocessing('I like cats.'))


# DataLoaderの作成

In [ ]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します
# https://github.com/YutaroOgawa/pytorch_advanced/issues/148
import torchtext 

# 文章とラベルの両方に用意します
max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# 引数の意味は次の通り
# init_token：全部の文章で、文頭に入れておく単語
# eos_token：全部の文章で、文末に入れておく単語


In [ ]:
# フォルダ「data」から各tsvファイルを読み込みます
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 動作確認
print('訓練および検証のデータ数', len(train_val_ds))
print('1つ目の訓練および検証のデータ', vars(train_val_ds[0]))


In [ ]:
import random
# torchtext.data.Datasetのsplit関数で訓練データとvalidationデータを分ける

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

# 動作確認
print('訓練データの数', len(train_ds))
print('検証データの数', len(val_ds))
print('1つ目の訓練データ', vars(train_ds[0]))


# ボキャブラリーを作成

In [ ]:
# It will take 2 minutes or so.
# torchtextで単語ベクトルとして英語学習済みモデルを読み込みます

from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')


# 単語ベクトルの中身を確認します
print("1単語を表現する次元数：", english_fasttext_vectors.dim)
print("単語数：", len(english_fasttext_vectors.itos))


In [ ]:
# ベクトル化したバージョンのボキャブラリーを作成します
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# ボキャブラリーのベクトルを確認します
print(TEXT.vocab.vectors.shape)  # 17916個の単語が300次元のベクトルで表現されている
TEXT.vocab.vectors

# ボキャブラリーの単語の順番を確認します
TEXT.vocab.stoi


In [ ]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=24, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=24, train=False, sort=False)


# 動作確認 検証データのデータセットで確認
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


このようにDataLoaderは単語のidを格納しているので、分散表現はディープラーニングモデル側でidに応じて取得してあげる必要があります。

ここまでの内容をフォルダ「utils」のdataloader.pyに別途保存しておき、次節からはこちらから読み込むようにします

以上

---
Revised by KAMEDA, Yoshinari at University of Tsukuba for lecture purpose.  
Original: https://github.com/YutaroOgawa/pytorch_advanced

2022/08/08.  
2021/08/03. 